In [1]:
import os
import sys
sys.path.append('..')  
import random
import pickle

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
from scipy.stats import hypergeom

In [4]:
from scregat.data_process import prepare_model_input, sum_counts, plot_edge, ATACGraphDataset
from run_scReGAT import *

/root/miniconda3/envs/scReGAT/lib/python3.8/site-packages/torch_geometric/typing.py:68: UserWarning: An issue occurred while importing 'pyg-lib'. Disabling its usage. Stacktrace: /lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.29' not found (required by /root/miniconda3/envs/scReGAT/lib/python3.8/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'pyg-lib'. "
/root/miniconda3/envs/scReGAT/lib/python3.8/site-packages/torch_geometric/typing.py:124: UserWarning: An issue occurred while importing 'torch-sparse'. Disabling its usage. Stacktrace: /lib/x86_64-linux-gnu/libm.so.6: version `GLIBC_2.29' not found (required by /root/miniconda3/envs/scReGAT/lib/python3.8/site-packages/libpyg.so)
  warnings.warn(f"An issue occurred while importing 'torch-sparse'. "


In [5]:
ATAC_h5ad_file = "../data/scATAC_MFG.h5ad"
RNA_h5ad_file = "../data/scRNA_MFG.h5ad"

In [6]:
adata_atac = sc.read_h5ad(ATAC_h5ad_file)
adata_atac

AnnData object with n_obs × n_vars = 2174 × 292156
    obs: 'celltype', 'celltype_rna'
    var: 'peak'

In [7]:
adata_rna = sc.read_h5ad(RNA_h5ad_file)
adata_rna

AnnData object with n_obs × n_vars = 15603 × 50281
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'cluster', 'class', 'brain_subregion', 'donor', 'sex', 'facs_sort_criteria', 'seq_batch', 'total_reads', 'percent_reads_unique', 'celltype'

In [8]:
adata_atac.obs.celltype.unique()

['inhibitory_neuron', 'oligodendrocytes', 'microglia', 'excitatory_neuron', 'astrocyte', 'polydendrocytes']
Categories (6, object): ['astrocyte', 'excitatory_neuron', 'inhibitory_neuron', 'microglia', 'oligodendrocytes', 'polydendrocytes']

In [9]:
adata_rna.obs.celltype.unique()

['inhibitory_neuron', 'excitatory_neuron', 'oligodendrocytes', 'OPC', 'astrocyte', 'microglia', 'polydendrocytes']
Categories (7, object): ['OPC', 'astrocyte', 'excitatory_neuron', 'inhibitory_neuron', 'microglia', 'oligodendrocytes', 'polydendrocytes']

In [10]:
adata_rna.obs['celltype'] = adata_rna.obs['celltype'].astype('object')
df_rna = sum_counts(adata_rna,by = 'celltype',marker_gene_num=300)

**finished identifying marker genes by COSG**


In [11]:
df_rna

,KHDRBS2,GRIK1-AS2,EBI3,RBFOX3,FSTL4,CLMN,HLA-DRA,SLC31A2,LINC00693,SMAD6,...,MMP16,LMO2,ZIC2,S1PR1,SLC24A3,MATN2,LOC105371450,LOC100289473,MALRD1,LOC105375946
celltype,,,,,,,,,,,,,,,,,,,,,
OPC,204.413307,25.355736,0.118630,3.951372,13.118019,8.731940,0.160989,10.183867,141.483908,0.289306,...,734.481218,12.977100,1.189517,0.632859,390.729394,158.011564,0.446021,2.449428,110.235446,0.005007
astrocyte,15.935032,0.005369,0.306355,17.003299,15.536367,95.553408,1.783011,11.943250,199.376715,18.062374,...,24.333656,39.471486,33.008490,54.069509,244.056471,29.233064,39.901926,154.940513,57.955775,0.005844
excitatory_neuron,30441.438663,26.970255,41.331078,15538.866045,15677.835472,621.224506,38.114010,451.525416,1770.539696,19.737002,...,19761.874766,73.867485,2.126392,6.050989,534.205947,1136.859327,83.222972,432.758441,801.517704,3.393977
inhibitory_neuron,7450.173362,943.230022,7.470723,4392.758670,756.419344,425.239518,10.876037,156.341741,3110.245013,14.016254,...,4018.684764,148.976759,0.375682,10.579516,4693.369108,602.001001,10.722428,88.373405,187.004622,2.663826
microglia,4.720456,0.006326,11.719547,0.501443,2.513341,2.532980,88.713636,0.556713,0.189775,11.052461,...,1.490889,20.653181,0.000000,0.005577,2.496617,1.170771,0.000000,1.102326,3.935479,13.484683
oligodendrocytes,69.578624,3.469668,0.862455,11.141468,16.277593,610.536332,4.084310,339.729936,17.202339,1.092756,...,19.026296,2.189157,0.200800,0.010993,8.541102,166.326730,0.760266,5.022869,22.311059,0.075826
polydendrocytes,3.151151,0.004475,0.000000,0.078543,0.560613,5.565757,0.086239,2.482783,2.527658,4.781364,...,0.637749,6.528295,2.625542,1.142909,0.006362,3.687775,0.013367,0.000000,0.044306,0.000000


In [12]:
dataset_atac = prepare_model_input(
    adata_atac=adata_atac,
    path_data_root = './' ,
    file_atac = ATAC_h5ad_file, 
    df_rna_celltype = df_rna,
    path_eqtl = '../data/all_tissue_SNP_Gene.txt',
    Hi_C_file_suffix = "_" + "brain",
    hg19tohg38 = False, min_percent = 0.01, use_additional_tf=True, tissue_cuttof=10)

only dataset_obj ...
processing Hi-C ...


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


processing TF ...
additional TF...
total candidate tf-gene:  28054


In [13]:
dataset_atac.list_graph[0]

Data(x=[61528, 1], edge_index=[2, 69180], y=[1], edge_tf=[155, 2], y_exp=[1348], cell='TCCATACCAATGATGCGGCATTCT')

In [15]:
file_atac_test = os.path.join('../data/', 'dataset_atac_core_MFG.pkl')
with open(file_atac_test, 'wb') as w_pkl:
    str_pkl = pickle.dumps(dataset_atac)
    w_pkl.write(str_pkl)

### Add Tissue-specific TF-gene 

In [16]:
df = pd.read_csv("../data/TF_Gene_tissue_Brain.csv", index_col=0)
df.columns = ['TF', 'TargetGene', 'tissue_count']

In [17]:
df_tf = df

In [18]:
gene_list = list(dataset_atac.df_rna.columns)

In [19]:
import itertools

In [20]:
set_gene = set(gene_list)
tf_base_filtered = df_tf[df_tf['TF'].isin(set_gene) & df_tf['TargetGene'].isin(set_gene)]
connections = [pair for pair in itertools.product(set_gene, set_gene) ]
gene_pair_base = connections
tf_map_gene = set(tf_base_filtered['TF'].unique())
target_map_gene = set(tf_base_filtered['TargetGene'].unique())
tf_base_tuples = set(zip(tf_base_filtered['TF'], tf_base_filtered['TargetGene']))
map_pair = tf_base_tuples.intersection(gene_pair_base)
map_pair_list = list(map_pair)
df_tf_new = pd.DataFrame(map_pair_list, columns=['TF', 'TargetGene'])
df_tf_all = pd.concat([df_tf_new, dataset_atac.df_tf])
df_tf_all = df_tf_all.drop_duplicates()

In [21]:
dataset_atac.df_tf = df_tf_all

In [22]:
# Create a dictionary to store the index of each element in dataset_atac.array_peak
peak_index_dict = {peak: idx for idx, peak in enumerate(dataset_atac.array_peak)}

# Initialize lists to store indices
index_1 = []
index_2 = []

# Iterate over 'TF' and 'TargetGene' columns in dataset_atac.df_tf
for k1, k2 in zip(dataset_atac.df_tf['TF'].values, dataset_atac.df_tf['TargetGene'].values):
    # Use the dictionary to quickly retrieve indices
    index_1.append(peak_index_dict[k1])
    index_2.append(peak_index_dict[k2])

# Stack the two index lists column-wise and convert to a PyTorch tensor
tf_edge_vec = torch.tensor(np.vstack([index_1, index_2]).T)

# Assign the TF edge tensor to the edge_tf attribute of each graph in the list
for t in dataset_atac.list_graph:
    t.edge_tf = tf_edge_vec


In [23]:
dataset_atac.list_graph[0]

Data(x=[61528, 1], edge_index=[2, 69180], y=[1], edge_tf=[4036, 2], y_exp=[1348], cell='TCCATACCAATGATGCGGCATTCT')

In [24]:
file_atac_test = os.path.join('../data/', 'dataset_atac_core_MFG.pkl')
with open(file_atac_test, 'wb') as w_pkl:
    str_pkl = pickle.dumps(dataset_atac)
    w_pkl.write(str_pkl)